In [1]:
import pandas as pd
import numpy as np

In [2]:
df_customers= pd.read_csv("/content/AWCustomers.csv")
df_sales= pd.read_csv("/content/AWSales.csv")


df_merged = pd.merge(df_customers, df_sales, on="CustomerID")

df_merged["BirthDate"] = pd.to_datetime(df_merged["BirthDate"], errors="coerce")
df_merged["Age"] = pd.Timestamp.now().year - df_merged["BirthDate"].dt.year

FileNotFoundError: [Errno 2] No such file or directory: '/content/AWCustomers.csv'

In [ ]:
df_merged.head()

In [ ]:
selected_features = [
    "Age", "Gender", "MaritalStatus", "Education", "Occupation",
    "HomeOwnerFlag", "NumberCarsOwned", "NumberChildrenAtHome",
    "TotalChildren", "YearlyIncome", "AvgMonthSpend", "BikeBuyer"
]
df_selected = df_merged[selected_features].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, KBinsDiscretizer, OneHotEncoder
import numpy as np

# (a) Handle missing values
df_selected = df_selected.dropna()

# (b) Normalize Age, Income, Spend
scaler = MinMaxScaler()
df_selected[["Age","YearlyIncome","AvgMonthSpend"]] = scaler.fit_transform(
    df_selected[["Age","YearlyIncome","AvgMonthSpend"]])

# (c) Discretize Age into 5 bins
binning = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
df_selected["Age_binned"] = binning.fit_transform(df_selected[["Age"]])

# (d) Standardize YearlyIncome
standardizer = StandardScaler()
df_selected[["YearlyIncome"]] = standardizer.fit_transform(df_selected[["YearlyIncome"]])


In [ ]:
# (e) One Hot Encoding for categorical vars
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded = encoder.fit_transform(df_selected[["Gender","MaritalStatus","Education","Occupation"]])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

# Final transformed dataset
df_final = pd.concat([df_selected.drop(["Gender","MaritalStatus","Education","Occupation"], axis=1).reset_index(drop=True),
                      encoded_df.reset_index(drop=True)], axis=1)

df_final.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jaccard

# Select two records
obj1 = df_final.iloc[0].values.reshape(1, -1)
obj2 = df_final.iloc[1].values.reshape(1, -1)

# Cosine Similarity
cos_sim = cosine_similarity(obj1, obj2)[0][0]

# Jaccard Similarity (binary/categorical attributes)
jacc_sim = 1 - jaccard(obj1[0], obj2[0])

# Simple Matching Coefficient
smc = np.mean(obj1[0] == obj2[0])

print("Cosine Similarity:", cos_sim)
print("Jaccard Similarity:", jacc_sim)
print("Simple Matching Coefficient:", smc)

# Correlation between YearlyIncome and AvgMonthSpend
corr = df_final["YearlyIncome"].corr(df_final["AvgMonthSpend"])
print("Correlation between YearlyIncome & AvgMonthSpend:", corr)